<a href="https://colab.research.google.com/github/v-pablo/Face-recognition-final-project/blob/main/final%20project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import cv2
import os
import numpy as np
import tensorflow as tf
import PIL
import PIL.Image

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten, Activation, Conv2D,MaxPooling2D,BatchNormalization
#from keras.constraints import maxnorm
from keras.optimizers import SGD


# Saving and loading model and weights
#from keras.models import model_from_json
#from keras.models import load_model

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from google.colab import drive
from sklearn.preprocessing import StandardScaler


In [3]:
from pandas.core.frame import dataclasses_to_dicts
# load and prepare data
people = ['s01', 's02', 's03', 's04', 's05', 's06', 's07', 's08', 's09', 's10',
's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20',
's21', 's22', 's23', 's24', 's25', 's26', 's27', 's28', 's29', 's30',
's31', 's32', 's33', 's34', 's35', 's36', 's37', 's38', 's39', 's40',

's41', 's42', 's43', 's44', 's45', 's46', 's47', 's48', 's49', 's50']

num_classes = 50
img_data_list = []
labels = []
valid_images = [".jpg",".gif",".png"]

In [36]:
for index, person in enumerate(people):
  #print(index)
  for img_path in os.listdir(dir_path):
    name, ext = os.path.splitext(img_path)
    if ext.lower() not in valid_images:
      continue
img_data = cv2.imread(dir_path + '/' + img_path)
print (img_data.size)

921600


In [28]:
# convert image to gray
img_data=cv2.cvtColor(img_data, cv2.COLOR_BGR2GRAY)
img_data=cv2.resize(img_data, (64, 64))
img_data_list.append(img_data)
labels.append(index)

print(img_data)

[[145 214 146 ... 228 116  97]
 [146 208 163 ... 228 124  73]
 [146 194 194 ... 223 128  53]
 ...
 [ 43  15  43 ... 107  72  23]
 [ 43  19  44 ... 106  80  23]
 [ 43  33  42 ... 105  86  23]]


In [7]:
#converts to np array format for easy processing
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')

In [10]:
# scale down(so easy to work with)
img_data /= 255.0
img_data= np.expand_dims(img_data, axis= 4)

In [11]:
# convert class labels to encoding(binary class matrix)
Y = np_utils.to_categorical(labels, num_classes)
# print(Y)

In [16]:
#Shuffle the dataset
x,y= shuffle(img_data, Y, random_state=2)



In [17]:
#Split the dataset
X_train, X_test, y_train, y_test= train_test_split(x, y, test_size= 0.34, random_state= 2)


ValueError: ignored

In [ ]:
# Defining the model
input_shape=img_data[0].shape
print(input_shape)

In [46]:
#cnn code
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=(64, 64, 1), data_format="channels_last"))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))

model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))

model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [47]:
# Compile model i.e model parameters
epochs = 35
lrate = 0.001
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#print(model.summary())

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [3]:
# Fit the model i.e training the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=10)

NameError: ignored

In [ ]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")